In [ ]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 200)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))


import os

os.environ["R_HOME"] = "/root/miniconda3/envs/R/lib/R"

import rpy2.robjects as objects
from rpy2.robjects.packages import importr

base = importr("base")
r_pROC = importr("pROC")
base._libPaths()[0]


def df_stats(df):
    from tabulate import tabulate

    print("\n***** Shape: ", df.shape, " *****\n")

    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()

    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(
        list_stat_val, columns=["Name", "Null", "Unique", "Dtypes"]
    )
    print(tabulate(df_stat_val, headers="keys", tablefmt="psql"))
    return df.head()

In [ ]:
df_ifr = pd.read_csv(
    "../CathAI/data/DeepCORO/CathReport_MHI_Merged/2017-2021_CathReport_first_value_in_interval_IFR.csv"
)

In [ ]:
## Define column where IFR was performed
import pandas as pd


def create_new_col(df):
    # List of column names to be checked
    col_names = [
        "D2",
        "D3",
        "RVG1",
        "RVG2",
        "S1",
        "Saphène ou mammaire",
        "bx",
        "diagonal",
        "dist_lad",
        "dist_lcx",
        "dist_rca",
        "lad",
        "lcx",
        "leftmain",
        "lvp",
        "marg_d",
        "mid_lad",
        "mid_rca",
        "om1",
        "om2",
        "om3",
        "pda",
        "posterolateral",
        "prox_rca",
    ]

    # Iterating over rows using DataFrame.apply() and lambda function
    df["ifr_performed"] = df.apply(
        lambda row: ", ".join([col for col in col_names if row[col] != -1.0]), axis=1
    )
    return df

In [ ]:
df = create_new_col(df_ifr)
df["contains_lad"] = df["ifr_performed"].str.contains("lad")
df["contains_lcx"] = df["ifr_performed"].str.contains("lcx")
df["contains_rca"] = df["ifr_performed"].str.contains("rca|pda|posterolateral")

In [ ]:
display(df_ifr.head(n=1))

In [ ]:
### Merge with DICOMS Extracted

df_dicoms = pd.read_csv(
    "../CathAI/data/DeepCORO/EXAMS_Extracted/2017-2021_dicom_extracted_mod.csv"
)

In [ ]:
# Identify common columns in df and df_dicoms, excluding 'path'
common_cols = [col for col in df.columns if col in df_dicoms.columns and col != "path"]

# Drop common columns from df_dicoms
df_dicoms_dropped = df_dicoms.drop(columns=common_cols)

# Perform left merge
df_merged = pd.merge(
    df, df_dicoms_dropped, left_on="DICOMPath", right_on="path", how="left"
)

In [ ]:
### Merge with views
df_angle_object = pd.read_csv(
    "../CathAI/data/DeepCORO/CATHAI_Extracted_Concatenated/DeepCORO_df_angle_object_dicom_2017-2021.csv"
)
### Drop Unnamed columns
df_angle_object = df_angle_object.loc[
    :, ~df_angle_object.columns.str.contains("^Unnamed")
]

In [ ]:
# Perform left merge
df_merged_angle_object = pd.merge(df_merged, df_angle_object, on="path", how="left")

In [ ]:
display(df_merged_angle_object.object_value.value_counts())

In [ ]:
display(df_merged_angle_object.contains_lad.value_counts())
display(df_merged_angle_object.contains_lcx.value_counts())
display(df_merged_angle_object.contains_rca.value_counts())

In [ ]:
#1425 examens avec IFR dans IVA, LCX ou RCA
display(df_f.grou

In [ ]:
df_f.to_csv("data/df_f.csv", index=False)
df_merged_angle_object.to_csv("data/df_merged_angle_object_IFR.csv", index=False)

Sure, here's a markdown table based on the labels' definitions:

| Label | Name         | 'primary' range | 'secondary' range |
|-------|--------------|-----------------|-------------------|
| 0     | RAO Cranial  | -60 to -15      | 15 to 50          |
| 1     | AP Cranial   | -15 to 15       | 15 to 50          |
| 2     | LAO Cranial  | 15 to 60        | 15 to 50          |
| 3     | RAO Straight | -60 to -15      | -15 to 15         |
| 4     | AP           | -15 to 15       | -15 to 15         |
| 5     | LAO Straight | 15 to 60        | -15 to 15         |
| 6     | RAO Caudal   | -60 to -15      | -50 to -15        |
| 7     | AP Caudal    | -15 to 15       | -50 to -15        |
| 8     | LAO Caudal   | 15 to 60        | -50 to -15        |
| 9     | LAO Lateral  | -110 to -70     | -15 to 15         |
| 10    | RAO Lateral  | 70 to 110       | -15 to 15         |
| 11    | Other        | Not applicable  | Not applicable    |

Note: The ranges in the 'primary' and 'secondary' columns are inclusive.

## Generate videos

In [ ]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 200)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))

from downloadAvi import extract_avi_metadata as avi_meta

In [ ]:
# df_f = pd.read_csv("data/df_f.csv")
df_merged_angle_object = pd.read_csv("data/df_merged_angle_object_IFR.csv")

In [ ]:
df_filtered = df_merged_angle_object[
    (df_merged_angle_object["contains_lad"] == True)
    | (df_merged_angle_object["contains_lcx"] == True)
    | (df_merged_angle_object["contains_rca"] == True)
]
df_filtered.to_csv("data/df_merged_angle_object_IFR_performed_2017-2021.csv")

In [ ]:
avi_meta.extract_avi_and_metadata(
    "data/df_merged_angle_object_IFR_performed_2017-2021.csv",
    data_type="ANGIO",
    destinationFolder="deepIFR/",
    dataFolder="data/",
)

# Preview Videos

In [ ]:
import pandas as pd
from tqdm import tqdm


def create_labels(df, primary, secondary, label_column, label2_column):
    label_list = []
    label2_list = []
    for _, row in tqdm(df.iterrows()):
        p = row[primary]
        s = row[secondary]
        if (-60 <= p <= -15) and (15 <= s <= 50):
            label_list.append(0)
            label2_list.append("RAO Cranial")
        elif (-15 <= p <= 15) and (15 <= s <= 50):
            label_list.append(1)
            label2_list.append("AP Cranial")
        elif (15 <= p <= 60) and (15 <= s <= 50):
            label_list.append(2)
            label2_list.append("LAO Cranial")
        elif (-60 <= p <= -15) and (-15 <= s <= 15):
            label_list.append(3)
            label2_list.append("RAO Straight")
        elif (-15 <= p <= 15) and (-15 <= s <= 15):
            label_list.append(4)
            label2_list.append("AP")
        elif (15 <= p <= 60) and (-15 <= s <= 15):
            label_list.append(5)
            label2_list.append("LAO Straight")
        elif (-60 <= p <= -15) and (-50 <= s <= -15):
            label_list.append(6)
            label2_list.append("RAO Caudal")
        elif (-15 <= p <= 15) and (-50 <= s <= -15):
            label_list.append(7)
            label2_list.append("AP Caudal")
        elif (15 <= p <= 60) and (-50 <= s <= -15):
            label_list.append(8)
            label2_list.append("LAO Caudal")
        elif (-110 <= p <= -70) and (-15 <= s <= 15):
            label_list.append(9)
            label2_list.append("LAO Lateral")
        elif (70 <= p <= 110) and (-15 <= s <= 15):
            label_list.append(10)
            label2_list.append("RAO Lateral")
        else:
            label_list.append(11)
            label2_list.append("Other")

    df[label_column] = label_list
    df[label2_column] = label2_list
    return df


def convert_seconds_to_time(df, time_column, new_time_column):
    """
    Converts the specified column from seconds to datetime format.

    Parameters:
    df : DataFrame
        The DataFrame which contains the column to be converted.
    time_column : str
        The name of the column to be converted.

    Returns:
    DataFrame
        The DataFrame with the converted column.
    """
    df[new_time_column] = pd.to_timedelta(df[time_column], unit="s")
    return df


def format_time(df, time_column):
    """
    Formats the specified timedelta column to 'hh:mm:ss' format.

    Parameters:
    df : DataFrame
        The DataFrame which contains the column to be formatted.
    time_column : str
        The name of the column to be formatted.

    Returns:
    DataFrame
        The DataFrame with the formatted column.
    """
    df[time_column] = (
        df[time_column]
        .dt.components[["hours", "minutes", "seconds"]]
        .astype(str)
        .agg(":".join, axis=1)
    )
    return df

In [ ]:
df_ifr_videos = pd.read_csv("data/df_merged_angle_object_IFR_performed_2017-2021.csv")

In [ ]:
df_ifr_videos.loc[df_ifr_videos["series_time"] < 0, "series_time"] = None
df_ifr_videos = convert_seconds_to_time(
    df_ifr_videos, "series_time", "series_time_human"
)
df_ifr_videos = format_time(df_ifr_videos, "series_time_human")

df_ifr_videos.loc[df_ifr_videos["study_time"] < 0, "study_time"] = None
df_ifr_videos = convert_seconds_to_time(df_ifr_videos, "study_time", "study_time_human")
df_ifr_videos = format_time(df_ifr_videos, "study_time_human")

In [ ]:
df_ifr_metadata = pd.read_csv(
    "data/df_merged_angle_object_IFR_performed_2017-2021.csv_metadata_extracted.csv"
)

In [ ]:
## merged df_ifr_videos and df_ifr_metadata on dicom_path and path but remove from df_ifr_metadata the column StudyInstanceUID
df_ifr_metadata = df_ifr_metadata.drop(
    columns=["StudyInstanceUID", "series_time", "study_time"]
)
df_ifr_videos_metadata = pd.merge(
    df_ifr_videos, df_ifr_metadata, right_on="dicom_path", left_on="path", how="inner"
)

## Remove columns starting with Unnamed df_ifr_videos_metadata
df_ifr_videos_metadata = df_ifr_videos_metadata.loc[
    :, ~df_ifr_videos_metadata.columns.str.contains("^Unnamed")
]

In [ ]:
df_ifr_videos_metadata = create_labels(
    df_ifr_videos_metadata,
    "primary_angle",
    "secondary_angle",
    "angle_true_label",
    "angle_true_string_label",
)
df_ifr_videos_metadata.to_csv(
    "data/df_merged_angle_object_IFR_performed_2017_with_metadata_and_angles.csv"
)

In [ ]:
display(df_ifr_videos_metadata.angle_value.value_counts())
display(df_ifr_videos_metadata.angle_true_label.value_counts())
display(df_ifr_videos_metadata.angle_true_string_label.value_counts())
display(df_ifr_videos_metadata.object_value.value_counts())

In [ ]:
### Select first 3 videos for RCA :
## For LAD select view AP Cranial, LAO Cranial or RAO Cranial
## For LCX sleect view RAO Straight, LAO Caudal, RAO Caudal, AP Caudal

### Generate videos

## Now create a function  taking a dataframe in and processing it as follows : where if the column [‘contains_rca’] is True then select first three rows for each “ExamID”;
## If the column [‘contain_lad’] is True, then select first three rows based on the earliest [’ series_time’]   for each “ExamID” where “angle_true_string_label” is either "AP Cranial, LAO Cranial or RAO Cranial “ and “object_vlaue” is 5.0
### if [‘contains_lcx’] is True then select first three rows based on the earliest [’ series_time’] where “angle_true_string_label” is either "RAO Straight, LAO Caudal, RAO Caudal, AP Caudal “ and “object_vlaue” is 5.0.


def process_dataframe(df):
    # Replace .0 and convert to timedelta
    # df['series_time'] = pd.to_timedelta(df['series_time'].str.replace('.0', ''))

    # df['series_time'] = pd.to_datetime(df['series_time'])

    # if 'contains_rca' is True then select first three rows based on the earliest 'series_time' for each “ExamID”
    rca_df = (
        df[(df["contains_rca"] == True) & (df["object_value"] == 9.0)]
        .groupby("ExamID")
        .apply(lambda x: x.nsmallest(3, "series_time"))
        .reset_index(drop=True)
    )

    # if 'contain_lad' is True, then select first three rows based on the earliest 'series_time'
    lad_conditions = (
        (df["contains_lad"] == True)
        & (
            df["angle_true_string_label"].isin(
                ["AP Cranial", "LAO Cranial", "RAO Cranial"]
            )
        )
        & (df["object_value"] == 5.0)
    )
    lad_df = (
        df.loc[lad_conditions]
        .groupby("ExamID")
        .apply(lambda x: x.nsmallest(3, "series_time"))
        .reset_index(drop=True)
    )

    # if 'contains_lcx' is True then select first three rows based on the earliest 'series_time'
    lcx_conditions = (
        (df["contains_lcx"] == True)
        & (
            df["angle_true_string_label"].isin(
                ["RAO Straight", "LAO Caudal", "RAO Caudal", "AP Caudal"]
            )
        )
        & (df["object_value"] == 5.0)
    )
    lcx_df = (
        df.loc[lcx_conditions]
        .groupby("ExamID")
        .apply(lambda x: x.nsmallest(3, "series_time"))
        .reset_index(drop=True)
    )

    # concatenate all dataframes
    result_df = pd.concat([rca_df, lad_df, lcx_df])

    return result_df

In [ ]:
result_df = process_dataframe(df_ifr_videos_metadata)

In [ ]:
# result_df.to_csv('data/df_merged_angle_object_IFR_performed_2017_with_metadata_and_angles_processed_filtered_3_videos.csv')
# exploded_df.to_csv('data/exploded_df_merged_angle_object_IFR_performed_2017_with_metadata_and_angles_processed_filtered_3_videos.csv')

In [ ]:
result_df = pd.read_csv(
    "data/df_merged_angle_object_IFR_performed_2017_with_metadata_and_angles_processed_filtered_3_videos.csv"
)
display(result_df.describe())

In [ ]:
import os
import tempfile

from IPython.display import HTML, Video, display
from ipywidgets import interactive
from moviepy.editor import VideoFileClip

# Create a temporary directory
temp_dir = tempfile.TemporaryDirectory()

# Select the first ExamID
exam_id = result_df["ExamID"].iloc[3300]

# Select the corresponding rows
rows = result_df[result_df["ExamID"] == exam_id]
video_data = rows[["FileName", "angle_true_string_label", "ifr_performed"]].values

# Convert .avi to .mp4 and save them in the temporary directory
for i in range(len(video_data)):
    clip = VideoFileClip(video_data[i][0])
    new_video_path = os.path.join(temp_dir.name, f"video_{i}.mp4")
    clip.write_videofile(new_video_path, codec="libx264")
    video_data[i][0] = new_video_path


def play_video(index):
    video_path, angle_label, ifr_performed = video_data[index]
    ifr_performed = ifr_performed.split(",")[
        0
    ]  # pick only the first value when multiple values are present
    ifr_value = rows.iloc[index][ifr_performed]
    display(
        HTML(f"<h2>Angle: {angle_label}, IFR: {ifr_performed}, Value: {ifr_value}</h2>")
    )
    display(Video(video_path, embed=True))


video_player = interactive(play_video, index=(0, len(video_data) - 1))

display(video_player)  # Manually display the widget

In [ ]:
# When you're done with the videos, you can clean up the temporary directory
temp_dir.cleanup()

In [ ]:
display(result_df.iloc[3300])

Assuming that your `result_df` DataFrame and the corresponding `ifr_performed` column look something like this:

```
| FileName | ifr_performed        | posterolateral | prox_rca |
|----------|----------------------|----------------|----------|
| video_1  | posterolateral       | 1.2            | 0.9      |
| video_2  | prox_rca             | 1.1            | 1.0      |
| video_3  | posterolateral,prox_rca | 1.3            | 1.1      |
```

And you want to end up with a DataFrame like this:

```
| FileName | ifr_performed | ifr  |
|----------|---------------|------|
| video_1  | posterolateral| 1.2  |
| video_2  | prox_rca      | 1.0  |
| video_3  | posterolateral| 1.3  |
| video_3  | prox_rca      | 1.1  |
```

You can achieve this with the following code:

```python
import pandas as pd

# This is a helper function to explode rows with multiple IFR performed
def explode_row(row):
    ifrs = row['ifr_performed'].split(',')
    for ifr in ifrs:
        new_row = row.copy()
        new_row['ifr_performed'] = ifr.strip()  # Removing any potential leading/trailing spaces
        new_row['ifr'] = row[ifr.strip()]      # Retrieving the corresponding value
        exploded_rows.append(new_row)

# Create an empty list to hold the exploded rows
exploded_rows = []

# Iterate over each row in the original DataFrame
result_df.apply(explode_row, axis=1)

# Create a new DataFrame from the list of exploded rows
exploded_df = pd.DataFrame(exploded_rows)

# Reset the index
exploded_df.reset_index(drop=True, inplace=True)
```

Now `exploded_df` is a new DataFrame where each row represents a unique video-segment where IFR was performed, with a new column `ifr` that stores the value of the corresponding IFR.


In [ ]:
import pandas as pd

# This is a helper function to explode rows with multiple IFR performed


def explode_row(row):
    ifrs = row["ifr_performed"].split(",")
    for ifr in ifrs:
        new_row = row.copy()
        new_row[
            "ifr_performed"
        ] = ifr.strip()  # Removing any potential leading/trailing spaces
        new_row["ifr"] = row[ifr.strip()]  # Retrieving the corresponding value
        exploded_rows.append(new_row)

In [ ]:
# Create an empty list to hold the exploded rows
exploded_rows = []

# Iterate over each row in the original DataFrame
result_df.apply(explode_row, axis=1)

# Create a new DataFrame from the list of exploded rows
exploded_df = pd.DataFrame(exploded_rows)

# Reset the index
exploded_df.reset_index(drop=True, inplace=True)

In [ ]:
def df_stats(df):
    from tabulate import tabulate

    print("\n***** Shape: ", df.shape, " *****\n")

    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()

    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(
        list_stat_val, columns=["Name", "Null", "Unique", "Dtypes"]
    )
    print(tabulate(df_stat_val, headers="keys", tablefmt="psql"))
    return df.head()


display(df_stats(exploded_df))
display(df_stats(result_df))

In [ ]:
exploded_df.ifr.value_counts()